<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>


In [142]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

import requests # library to handle requests

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>


## 1. Web Scraping and Dataset Exploration


In [143]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [144]:
df = df[df.Borough != 'Not assigned']

In [145]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [146]:
df.shape

(103, 3)

NOTE: Most of the requirements were not applicable based on the current version of the table in the Wikipedia page.

## 2. Coordinates

In [147]:
# !conda install -c conda-forge geocoder

In [148]:
# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

Second option is taken: importing .csv file that contains the coordinates of the differents Postal Codes of Toronto Neighbourhoods.

In [149]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [150]:
coord_df = df_data_1
coord_df.shape

(103, 3)

In [151]:
df = pd.merge(df, coord_df, on = "Postal Code")
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## 3. Clustering

In [152]:
import json # library to handle JSON files

!pip3 install geopy # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip3 install folium # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [153]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top

In [154]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_Toronto)  
map_Toronto

In [155]:
df_Toronto_boroughs = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_Toronto_boroughs

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [156]:
address = 'Queen\'s Park, TO'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.659659, -79.3903399.


In [157]:
# create map of Toronto boroughs using latitude and longitude values
map_Toronto_boroughs = folium.Map(location = [latitude, longitude], zoom_start = 11)

# add markers to map
for lat, lng, label in zip(df_Toronto_boroughs['Latitude'], df_Toronto_boroughs['Longitude'], df_Toronto_boroughs['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto_boroughs)
map_Toronto_boroughs

#### Define Foursquare Credentials and Version

In [158]:
CLIENT_ID = 'QCI0HK4NH4VHBP2SMJCAR0VJFQKN54PGIUFPQZ0W5YMLL1C1' # your Foursquare ID
CLIENT_SECRET = 'YOXAKOKXEEHSUZCKDRJMXUKCLHUUE4BVLIYZMGZVKBYNLXXG' # your Foursquare Secret
VERSION = '20201212' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QCI0HK4NH4VHBP2SMJCAR0VJFQKN54PGIUFPQZ0W5YMLL1C1
CLIENT_SECRET:YOXAKOKXEEHSUZCKDRJMXUKCLHUUE4BVLIYZMGZVKBYNLXXG


### 3.1. Explore the Neighbourhoods in Toronto

Function to get all venues nearby a location in the Toronto boroughs:

In [186]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [187]:
toronto_boroughs_venues = getNearbyVenues(names = df_Toronto_boroughs['Neighbourhood'],
                                   latitudes = df_Toronto_boroughs['Latitude'],
                                   longitudes = df_Toronto_boroughs['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [188]:
print(toronto_boroughs_venues.shape)
toronto_boroughs_venues.head()

(1615, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [189]:
toronto_boroughs_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,61,61,61,61,61,61
Christie,16,16,16,16,16,16
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [190]:
print('There are {} uniques categories.'.format(len(toronto_boroughs_venues['Venue Category'].unique())))

There are 234 uniques categories.


### 3.2. Analyze each neighbourhood

In [191]:
# one hot encoding
toronto_boroughs_onehot = pd.get_dummies(toronto_boroughs_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_boroughs_onehot['Neighborhood'] = toronto_boroughs_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_boroughs_onehot.columns[-1]] + list(toronto_boroughs_onehot.columns[:-1])
toronto_boroughs_onehot = toronto_boroughs_onehot[fixed_columns]

toronto_boroughs_onehot.sample(10)

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
259,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
935,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
699,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1520,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1309,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1386,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
129,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1256,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [192]:
toronto_boroughs_onehot.shape

(1615, 234)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [193]:
toronto_boroughs_grouped = toronto_boroughs_onehot.groupby('Neighborhood').mean().reset_index()
toronto_boroughs_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.016393,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.016393,0.000000,0.000000,0.016393,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.025316,0.000000,0.000000,0.000000,0.0,0.000000,0.012658,0.000000,0.00,...,0.012658,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.012658,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.030303,0.000000,0.00,...,0.000000,0.000000,0.030303,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [194]:
toronto_boroughs_grouped.shape

(39, 234)

Let's print each neighborhood along with the top 5 most common venues

In [195]:
num_top_venues = 5

for hood in toronto_boroughs_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_boroughs_grouped[toronto_boroughs_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.05
2     Cheese Shop  0.04
3  Farmers Market  0.04
4          Bakery  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1  Breakfast Spot  0.08
2       Nightclub  0.08
3     Coffee Shop  0.08
4             Gym  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.12
1          Comic Shop  0.06
2       Auto Workshop  0.06
3          Skate Park  0.06
4      Farmers Market  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.20
1    Airport Lounge  0.13
2  Airport Terminal  0.13
3          Boutique  0.07
4           Airport  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.20
1               

Let's put that into a _pandas_ dataframe. First, let's write a function to sort the venues in descending order:

In [196]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [197]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_boroughs_grouped['Neighborhood']

for ind in np.arange(toronto_boroughs_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_boroughs_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Bakery,Beer Bar,Restaurant,Seafood Restaurant,Park,Liquor Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Nightclub,Gym,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Brewery,Garden Center,Fast Food Restaurant,Farmers Market,Auto Workshop,Spa,Burrito Place,Garden,Pizza Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Airport,Airport Food Court,Sculpture Garden,Boutique,Harbor / Marina,Rental Car Location,Boat or Ferry
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Salad Place,Thai Restaurant,Bubble Tea Shop,Donut Shop,Seafood Restaurant


### 3.3. Cluster Neighbourhoods

Run _k_-means to cluster the neighborhood into 5 clusters.

In [198]:
# set number of clusters
kclusters = 5

toronto_boroughs_grouped_clustering = toronto_boroughs_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_boroughs_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [199]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_boroughs_merged = df_Toronto_boroughs

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_boroughs_merged = toronto_boroughs_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_boroughs_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Theater,Pub,Farmers Market,French Restaurant,Gym / Fitness Center
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Gym,Fast Food Restaurant,Portuguese Restaurant,Park,Nightclub,Music Venue,Mexican Restaurant,Japanese Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Hotel,Bubble Tea Shop,Cosmetics Shop,Middle Eastern Restaurant,Bookstore,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Gastropub,Cocktail Bar,American Restaurant,Gym,Park,Seafood Restaurant,Restaurant,Italian Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Trail,Health Food Store,Wine Shop,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant


Finally, let's visualize the resulting clusters

In [200]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_boroughs_merged['Latitude'], toronto_boroughs_merged['Longitude'], toronto_boroughs_merged['Neighbourhood'], toronto_boroughs_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)   
map_clusters

### 3.4. Examine Clusters

#### CLUSTER 1

In [212]:
toronto_boroughs_merged.loc[toronto_boroughs_merged['Cluster Labels'] == 0, toronto_boroughs_merged.columns[[2] + list(range(5, toronto_boroughs_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,0,Ice Cream Shop,Garden,Health & Beauty Service,Home Service,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store,Wine Shop


This is the **Ice-Cream and Gardening** Neighbourhood of Toronto.

#### CLUSTER 2

In [213]:
toronto_boroughs_merged.loc[toronto_boroughs_merged['Cluster Labels'] == 1, toronto_boroughs_merged.columns[[2] + list(range(5, toronto_boroughs_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",1,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Theater,Pub,Farmers Market,French Restaurant,Gym / Fitness Center
1,"Queen's Park, Ontario Provincial Government",1,Coffee Shop,Sushi Restaurant,Gym,Fast Food Restaurant,Portuguese Restaurant,Park,Nightclub,Music Venue,Mexican Restaurant,Japanese Restaurant
2,"Garden District, Ryerson",1,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Hotel,Bubble Tea Shop,Cosmetics Shop,Middle Eastern Restaurant,Bookstore,Ramen Restaurant
3,St. James Town,1,Coffee Shop,Café,Gastropub,Cocktail Bar,American Restaurant,Gym,Park,Seafood Restaurant,Restaurant,Italian Restaurant
4,The Beaches,1,Pub,Trail,Health Food Store,Wine Shop,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant
5,Berczy Park,1,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Bakery,Beer Bar,Restaurant,Seafood Restaurant,Park,Liquor Store
6,Central Bay Street,1,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Salad Place,Thai Restaurant,Bubble Tea Shop,Donut Shop,Seafood Restaurant
7,Christie,1,Grocery Store,Café,Park,Restaurant,Candy Store,Baby Store,Coffee Shop,Nightclub,Athletics & Sports,Italian Restaurant
8,"Richmond, Adelaide, King",1,Coffee Shop,Café,Restaurant,Gym,Clothing Store,Thai Restaurant,Hotel,Deli / Bodega,Cosmetics Shop,Bookstore
9,"Dufferin, Dovercourt Village",1,Bakery,Pharmacy,Brewery,Athletics & Sports,Bank,Supermarket,Middle Eastern Restaurant,Café,Pool,Bar


These are the **all-round** neighbourhoods of Toronto.

#### CLUSTER 3

In [214]:
toronto_boroughs_merged.loc[toronto_boroughs_merged['Cluster Labels'] == 2, toronto_boroughs_merged.columns[[2] + list(range(5, toronto_boroughs_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park, Summerhill East",2,Summer Camp,Wine Shop,Falafel Restaurant,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


This is the **Summer Camp and Wine** Neighbourhood of Toronto.

#### CLUSTER 4

In [215]:
toronto_boroughs_merged.loc[toronto_boroughs_merged['Cluster Labels'] == 3, toronto_boroughs_merged.columns[[2] + list(range(5, toronto_boroughs_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,"Forest Hill North & West, Forest Hill Road Park",3,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Wine Shop,Deli / Bodega,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


This is the **Jewelry and Trailing** Neighbourhood of Toronto.

#### CLUSTER 5

In [216]:
toronto_boroughs_merged.loc[toronto_boroughs_merged['Cluster Labels'] == 4, toronto_boroughs_merged.columns[[2] + list(range(5, toronto_boroughs_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Rosedale,4,Park,Playground,Trail,Cupcake Shop,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


This is the **Outdoors** Neighbourhood of Toronto.